In [ ]:
import cv2
import mediapipe as mp

# Initialize MediaPipe pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,model_complexity=2, enable_segmentation=True)

# Load your video
cap = cv2.VideoCapture('head-pose-face-detection-female.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame for pose detection
    results = pose.process(frame_rgb)

    # Draw pose annotations on the frame
    annotated_image = frame.copy()
    mp.solutions.drawing_utils.draw_landmarks(
        annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow('Pose', annotated_image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()

In [4]:
import cv2
import torch
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from midas.model_loader import load_model


ModuleNotFoundError: No module named 'midas'

In [5]:

# Load MiDaS model for depth estimation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
midas, transform, net_w, net_h = load_model(
    "DPT_Large", device, optimize=False)

# Initialize MediaPipe pose model
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=2, enable_segmentation=True)

# Load your video
cap = cv2.VideoCapture('path_to_video.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Pose estimation
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    # Depth estimation
    input_batch = transform({"image": frame_rgb / 255.0}
                            )["image"].to(device).unsqueeze(0)
    with torch.no_grad():
        prediction = midas(input_batch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=frame_rgb.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()
    depth = prediction.cpu().numpy()

    # Draw pose annotations
    annotated_image = frame.copy()
    mp.solutions.drawing_utils.draw_landmarks(
        annotated_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow('Pose', annotated_image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'load_model' is not defined